In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from keras import utils
from xgboost import XGBRegressor
import xgboost as xgb
import xgboost


In [2]:
print(xgboost.__version__)

2.0.3


In [3]:
# Read csv
df = pd.read_csv('commodity 2000-2022.csv')

In [4]:
# Replace string values into integers
# In integer values:
# Gold = 0, Palladium = 1, Nickel = 2, Brent Oil = 3, Natural Gas = 4 and US Wheat = 5
df = df.replace({'Gold': 0, 'Palladium': 1, 'Nickel': 2, 'Brent Oil': 3, 'Natural Gas': 4, 'US Wheat': 5})

C:\Users\vilil\AppData\Local\Temp\ipykernel_19196\363166622.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Gold': 0, 'Palladium': 1, 'Nickel': 2, 'Brent Oil': 3, 'Natural Gas': 4, 'US Wheat': 5})


In [5]:
# convert the date column into a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# extract the day, month, and year components
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# show the modified data frame
print(df)

       Symbol       Date     Open     High      Low    Close  Volume  Day  \
0           0 2000-01-04   281.00   281.00   281.00   282.70       4    4   
1           0 2000-01-05   283.20   283.20   283.20   281.10      16    5   
2           0 2000-01-06   281.40   281.40   281.40   281.40       0    6   
3           0 2000-01-07   281.90   281.90   281.90   281.90       0    7   
4           0 2000-01-10   281.70   281.70   281.70   281.70       0   10   
...       ...        ...      ...      ...      ...      ...     ...  ...   
29930       5 2022-04-04   989.50  1014.50   984.75  1010.25       0    4   
29931       5 2022-04-05  1020.50  1074.00  1020.00  1045.25       0    5   
29932       5 2022-04-06  1049.40  1056.40  1024.25  1041.12       0    6   
29933       5 2022-04-07  1040.25  1044.50  1017.75  1020.00       0    7   
29934       5 2022-04-08  1020.00  1054.25  1017.00  1051.50       0    8   

       Month  Year  
0          1  2000  
1          1  2000  
2          1

In [6]:
# drop unneccesary "Date" because now on we'll use it in day, month and year sections
df = df.drop('Date', axis=1)

print(df)

       Symbol     Open     High      Low    Close  Volume  Day  Month  Year
0           0   281.00   281.00   281.00   282.70       4    4      1  2000
1           0   283.20   283.20   283.20   281.10      16    5      1  2000
2           0   281.40   281.40   281.40   281.40       0    6      1  2000
3           0   281.90   281.90   281.90   281.90       0    7      1  2000
4           0   281.70   281.70   281.70   281.70       0   10      1  2000
...       ...      ...      ...      ...      ...     ...  ...    ...   ...
29930       5   989.50  1014.50   984.75  1010.25       0    4      4  2022
29931       5  1020.50  1074.00  1020.00  1045.25       0    5      4  2022
29932       5  1049.40  1056.40  1024.25  1041.12       0    6      4  2022
29933       5  1040.25  1044.50  1017.75  1020.00       0    7      4  2022
29934       5  1020.00  1054.25  1017.00  1051.50       0    8      4  2022

[29935 rows x 9 columns]


In [7]:
# Find out if dataset contains any missing values
df.isna().sum()

Symbol    0
Open      0
High      0
Low       0
Close     0
Volume    0
Day       0
Month     0
Year      0
dtype: int64

In [8]:
# Find out if dataset contains any duplicated values
df.duplicated().sum()

0

In [9]:
# a common trick in X/y -split
# X = everything else in the DataFrame minus the target variable
# y = only the target variable
X = df.drop('Symbol', axis=1)
y = df['Symbol']

In [10]:
# version 1, SciPy, extreme outliers that go under -3 or over +3 in normal distribution
from scipy import stats
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

# version 2, use quantiles
 #this removes top 10% and bottom 10% from the distribution (extreme values)
 #price is used as the variable. you can also use this for other variables, like area
#df_middle= df.query('Close >= Close.quantile(q=0.40) and Close < Close.quantile(q=0.60)')

In [11]:
# REMEMBER TO SCALE THE TARGET VARIABLE (y)
# WITH MINMAXSCALER BEFORE DOING THE X/y -split

# initalize scaler
scalerY = MinMaxScaler()

# name of target variable
target_variable = "Symbol"

# save min/max -values of target value
# for bettermetrics later
min_y = df[target_variable].min()
max_y = df[target_variable].max()

# scale y-variable
num_vars = [target_variable]
df[num_vars] = scalerY.fit_transform(df[num_vars])

<b>Train/Test splits</b>

In [12]:
df

,Symbol,Open,High,Low,Close,Volume,Day,Month,Year
0,0.0,281.00,281.00,281.00,282.70,4,4,1,2000
1,0.0,283.20,283.20,283.20,281.10,16,5,1,2000
2,0.0,281.40,281.40,281.40,281.40,0,6,1,2000
3,0.0,281.90,281.90,281.90,281.90,0,7,1,2000
4,0.0,281.70,281.70,281.70,281.70,0,10,1,2000
...,...,...,...,...,...,...,...,...,...
29930,1.0,989.50,1014.50,984.75,1010.25,0,4,4,2022
29931,1.0,1020.50,1074.00,1020.00,1045.25,0,5,4,2022
29932,1.0,1049.40,1056.40,1024.25,1041.12,0,6,4,2022
29933,1.0,1040.25,1044.50,1017.75,1020.00,0,7,4,2022


In [13]:
from sklearn.preprocessing import OrdinalEncoder

X, y = df.drop("Symbol", axis=1), df[['Symbol']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

In [14]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [19]:
# Create classification matrices
#dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
#dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [25]:

# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": 5}
n = 1000

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

results = xgb.cv(
    tree_method = "hist",
    device = "cuda",
    params=params,
    dtrain=dtrain_clf,
    num_boost_round=n,
    nfold=5,
    metrics=["mlogloss", "auc", "merror"],
)

XGBoostError: [22:37:49] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\multiclass_obj.cu:123: SoftmaxMultiClassObj: label must be in [0, num_class).